In [1]:
# 0. 外部ライブラリ
!pip -q install mahjong pytest requests beautifulsoup4

# 1. Google Drive をマウント（メッセージは無視して OK）
from google.colab import drive
drive.mount('/content/drive')

# 2. パッケージ用ディレクトリを作成
package_dir = '/content/drive/MyDrive/mahjong_py'   # ← ここを変更
import os, sys, pathlib
os.makedirs(package_dir, exist_ok=True)

# 3. GitHub からファイルを取得
files = [   # リポジトリに存在する全 *.py を列挙
    "__init__.py","analyzer.py","converters.py","display_agari_fixed.py",
    "display_call.py","display_discard_hand_at.py","display_dora_fixed.py",
    "display_handflow.py","display_reach_fixed.py","display_ryuukyoku.py",
    "parser.py","shanten_calc.py","splitter.py",
    # tests
    "test_analyzer.py","test_converters.py","test_display_agari_fixed.py",
    "test_display_call.py","test_display_discard_hand_at.py","test_display_dora_fixed.py",
    "test_display_handflow.py","test_display_reach_fixed.py","test_display_ryuukyoku_info.py",
    "test_init.py","test_parser.py","test_shanten_calc.py","test_splitter.py"
]
base_url = "https://raw.githubusercontent.com/furapotedesu/tenhouhaihu/main/"
for f in files:
    !wget -q {base_url}{f} -O {package_dir}/{f}

# 4. import 検索パスに “Drive 直下” を追加
sys.path.append('/content/drive/MyDrive')



# 5. pytest 実行
!PYTHONPATH="/content/drive/MyDrive" pytest {package_dir} --maxfail=1 --disable-warnings -q


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.9/60.9 kB 2.5 MB/s eta 0:00:00
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
................................................                         [100%]
48 passed in 10.66s


In [2]:
import importlib.util, sys, os

# ❶ モジュール格納先を定義
my_modules_path = '/content/drive/MyDrive/my_modules'

# ❷ 動的インポート
module_path = os.path.join(my_modules_path, 'display_discard_hand_at.py')
spec = importlib.util.spec_from_file_location('display_discard_hand_at', module_path)
display_module = importlib.util.module_from_spec(spec)
sys.modules['display_discard_hand_at'] = display_module
spec.loader.exec_module(display_module)

# ❸ 必要な関数を取得
display_discard_hand_at = display_module.display_discard_hand_at


In [6]:
from mahjong_py.display_discard_hand_at import (
    parse_player_names, get_kyoku_segment, process_segment
)
from mahjong_py.converters import convert_tile_id_to_str
from mahjong_py.shanten_calc import calculate_shanten

def display_discard_with_shanten(game_str: str, kyoku_index: int) -> None:
    """
    game_str: 天鳳MJログの文字列
    kyoku_index: 0 始まりの何局目か
    """
    # 丸囲み数字を半角数字に置換するマップ
    circled_to_digit = {
        '①':'1','②':'2','③':'3','④':'4','⑤':'5',
        '⑥':'6','⑦':'7','⑧':'8','⑨':'9'
    }
    def normalize_tile(tile: str) -> str:
        # 各文字をマップで置換（それ以外はそのまま）
        return ''.join(circled_to_digit.get(ch, ch) for ch in tile)

    name_map = parse_player_names(game_str)
    segment = get_kyoku_segment(game_str, kyoku_index)
    data = process_segment(segment)

    for entry in data:
        # タプルの長さをチェックして安全にアンパック
        if len(entry) == 4:
            player, count, hand_ids, meld_list = entry
        elif len(entry) == 3:
            player, count, hand_ids = entry
            meld_list = []
        else:
            raise ValueError(f"予期せぬタプル長: {len(entry)} 要素 in {entry!r}")

        name = name_map.get(player, f"Player{player}")
        # 隠れ牌を日本語表記
        hidden = [convert_tile_id_to_str(t) for t in hand_ids]

        # 副露込みのサンプル手牌リストを作成
        sample_hand = hidden.copy()
        for mtype, tiles in meld_list:
            sample_hand.extend(convert_tile_id_to_str(t) for t in tiles)

        # まず丸囲み数字を半角に変換してからシャンテン計算
        sample_hand_norm = [normalize_tile(t) for t in sample_hand]
        melds_count = len(meld_list)
        shanten = calculate_shanten(sample_hand_norm, melds_count)

        # 表示
        print(f"{name} さんの {count} 回目の打牌後の手牌 (隠れ牌): {hidden}")
        if meld_list:
            melds_str = [
                f"{mtype}:{[convert_tile_id_to_str(t) for t in tiles]}"
                for mtype, tiles in meld_list
            ]
            print(f"  副露メンツ: {melds_str}")
        print(f"  シャンテン数: {shanten}\n")


In [3]:
from mahjong_py.splitter import split_games

# ログファイルを読み込み、ゲームを分割
with open("/content/drive/MyDrive/mahjong_data/log_analysis/luckyj_raw.txt", "r", encoding="utf-8") as f:
    raw_text = f.read()
games = split_games(raw_text)

# 2番目のゲーム（インデックス1）を抽出
game2_str = games[1]

In [5]:
display_discard_with_shanten(game2_str, 6)


DEBUG:mahjong_py.analyzer:[DEBUG] decode_mentsu(0x7f01) → [126, 127, 124, 125], type=daiminkan
DEBUG:mahjong_py.analyzer:[DEBUG] decode_mentsu(0xae49) → [116, 117, 119], type=pon
DEBUG:mahjong_py.analyzer:[DEBUG] decode_mentsu(0x6869) → [69, 68, 70], type=pon


心 さんの 1 回目の打牌後の手牌 (隠れ牌): ['發', '白', '七萬', '⑧筒', '②筒', '⑥筒', '一萬', '②筒', '八萬', '2索', '東', '四萬', '二萬']
  シャンテン数: {'通常': 4, '七対子': 6, '国士無双': 10}

牌操作万歳！ さんの 1 回目の打牌後の手牌 (隠れ牌): ['⑥筒', '東', '⑧筒', '1索', '5索', '3索', '東', '6索', '中', '南', '⑤筒', '一萬', '⑦筒']
  シャンテン数: {'通常': 3, '七対子': 6, '国士無双': 8}

ⓝLuckyJ さんの 1 回目の打牌後の手牌 (隠れ牌): ['③筒', '1索', '南', '五萬', '9索', '③筒', '六萬', '③筒', '六萬', '三萬', '④筒', '①筒', '1索']
  シャンテン数: {'通常': 3, '七対子': 4, '国士無双': 9}

こうえい さんの 1 回目の打牌後の手牌 (隠れ牌): ['白', '2索', '中', '西', '白', '2索', '⑨筒', '赤⑤筒', '⑧筒', '⑨筒', '4索', '西', '4索']
  シャンテン数: {'通常': 3, '七対子': 2, '国士無双': 9}

心 さんの 2 回目の打牌後の手牌 (隠れ牌): ['白', '七萬', '⑧筒', '②筒', '⑥筒', '一萬', '②筒', '八萬', '2索', '東', '四萬', '二萬', '赤五萬']
  シャンテン数: {'通常': 3, '七対子': 6, '国士無双': 11}

牌操作万歳！ さんの 2 回目の打牌後の手牌 (隠れ牌): ['⑥筒', '東', '⑧筒', '1索', '5索', '3索', '東', '6索', '中', '南', '⑤筒', '一萬', '⑦筒']
  シャンテン数: {'通常': 3, '七対子': 6, '国士無双': 8}

ⓝLuckyJ さんの 2 回目の打牌後の手牌 (隠れ牌): ['③筒', '1索', '南', '五萬', '9索', '③筒', '六萬', '③筒', '六萬', '三萬', '④筒', '1索', '八萬']
  シャンテン数: {

In [14]:
display_discard_hand_at(game2_str, 6)

DEBUG:mahjong_py.analyzer:[DEBUG] decode_mentsu(0x7f01) → [126, 127, 124, 125], type=daiminkan
DEBUG:mahjong_py.analyzer:[DEBUG] decode_mentsu(0xae49) → [116, 117, 119], type=pon
DEBUG:mahjong_py.analyzer:[DEBUG] decode_mentsu(0x6869) → [69, 68, 70], type=pon


心 さんの 1 回目の打牌後の手牌 (隠れ牌): ['發', '白', '七萬', '⑧筒', '②筒', '⑥筒', '一萬', '②筒', '八萬', '2索', '東', '四萬', '二萬']
牌操作万歳！ さんの 1 回目の打牌後の手牌 (隠れ牌): ['⑥筒', '東', '⑧筒', '1索', '5索', '3索', '東', '6索', '中', '南', '⑤筒', '一萬', '⑦筒']
ⓝLuckyJ さんの 1 回目の打牌後の手牌 (隠れ牌): ['③筒', '1索', '南', '五萬', '9索', '③筒', '六萬', '③筒', '六萬', '三萬', '④筒', '①筒', '1索']
こうえい さんの 1 回目の打牌後の手牌 (隠れ牌): ['白', '2索', '中', '西', '白', '2索', '⑨筒', '赤⑤筒', '⑧筒', '⑨筒', '4索', '西', '4索']
心 さんの 2 回目の打牌後の手牌 (隠れ牌): ['白', '七萬', '⑧筒', '②筒', '⑥筒', '一萬', '②筒', '八萬', '2索', '東', '四萬', '二萬', '赤五萬']
牌操作万歳！ さんの 2 回目の打牌後の手牌 (隠れ牌): ['⑥筒', '東', '⑧筒', '1索', '5索', '3索', '東', '6索', '中', '南', '⑤筒', '一萬', '⑦筒']
ⓝLuckyJ さんの 2 回目の打牌後の手牌 (隠れ牌): ['③筒', '1索', '南', '五萬', '9索', '③筒', '六萬', '③筒', '六萬', '三萬', '④筒', '1索', '八萬']
こうえい さんの 2 回目の打牌後の手牌 (隠れ牌): ['白', '2索', '中', '西', '白', '2索', '⑨筒', '赤⑤筒', '⑨筒', '4索', '西', '4索', '白']
心 さんの 3 回目の打牌後の手牌 (隠れ牌): ['七萬', '⑧筒', '②筒', '⑥筒', '一萬', '②筒', '八萬', '2索', '東', '四萬', '二萬', '赤五萬', '西']
こうえい さんの 3 回目の打牌後の手牌 (隠れ牌): ['2索', '西', '2索', '⑨筒', '赤⑤筒', '

In [15]:
from mahjong_py.display_call import display_calls_fixed

display_calls_fixed(game2_str, game_index=2)


DEBUG:mahjong_py.analyzer:[DEBUG] decode_mentsu(0x2a29) → [28, 30, 31], type=pon
DEBUG:mahjong_py.display_call:[DEBUG] Before removal: who=0, hand=[1, 5, 24, 25, 26, 30, 31, 106, 107, 114, 115, 126, 127]
DEBUG:mahjong_py.display_call:[DEBUG] Meld tiles: [28, 30, 31], type: pon
DEBUG:mahjong_py.display_call:[DEBUG] After removal: temp_hand=[1, 5, 24, 25, 26, 106, 107, 114, 115, 126, 127]
DEBUG:mahjong_py.analyzer:[DEBUG] decode_mentsu(0x1800) → [24, 25, 26, 27], type=ankan
DEBUG:mahjong_py.display_call:[DEBUG] Before removal: who=0, hand=[1, 24, 25, 26, 27, 106, 107, 114, 115, 126, 127]
DEBUG:mahjong_py.display_call:[DEBUG] Meld tiles: [24, 25, 26, 27], type: ankan
DEBUG:mahjong_py.display_call:[DEBUG] After removal: temp_hand=[1, 106, 107, 114, 115, 126, 127]
DEBUG:mahjong_py.analyzer:[DEBUG] decode_mentsu(0xa22a) → [108, 110, 111], type=pon
DEBUG:mahjong_py.display_call:[DEBUG] Before removal: who=3, hand=[1, 4, 21, 25, 26, 30, 62, 64, 69, 81, 110, 111, 134]
DEBUG:mahjong_py.display_c



└─ Game 2 鳴き情報 ┐
Game 2 第1局 東家
   ツモ数: 5
   鳴き内容: 八萬 をポンして 八萬 八萬 と組み合わせた
   直後の打牌: 二萬
   直後の手牌: 9索 9索 一萬 七萬 七萬 七萬 南 南 白 白
Game 2 第1局 東家
   ツモ数: 10
   鳴き内容: 七萬 を暗槓した（七萬 七萬 七萬 七萬）
   直後の打牌: ①筒
   直後の手牌: 9索 9索 一萬 南 南 白 白
Game 2 第3局 北家
   ツモ数: 7
   鳴き内容: 東 をポンして 東 東 と組み合わせた
   直後の打牌: 七萬
   直後の手牌: 3索 ⑦筒 ⑧筒 ⑨筒 一萬 七萬 中 二萬 八萬 六萬
Game 2 第3局 南家
   ツモ数: 11
   鳴き内容: 六萬 をチーして 七萬 赤五萬 と組み合わせた
   直後の打牌: ⑤筒
   直後の手牌: 2索 3索 4索 6索 7索 ③筒 ④筒 ⑤筒 ⑥筒 ⑥筒
Game 2 第4局 東家
   ツモ数: 6
   鳴き内容: 1索 を暗槓した（1索 1索 1索 1索）
   直後の打牌: 9索
   直後の手牌: 2索 6索 6索 7索 ⑤筒 ⑥筒 ⑦筒 三萬 赤五萬
Game 2 第4局 北家
   ツモ数: 11
   鳴き内容: 西 をポンして 西 西 と組み合わせた
   直後の打牌: 中
   直後の手牌: 5索 5索 ⑥筒 ⑥筒 七萬 二萬 二萬 二萬 五萬 赤5索
Game 2 第6局 南家
   ツモ数: 10
   鳴き内容: 二萬 をチーして 三萬 四萬 と組み合わせた
   直後の打牌: 白
   直後の手牌: 2索 4索 ②筒 ③筒 ③筒 ⑦筒 ⑦筒 七萬 五萬 六萬
Game 2 第6局 南家
   ツモ数: 13
   鳴き内容: ③筒 をチーして ④筒 ②筒 と組み合わせた
   直後の打牌: ③筒
   直後の手牌: 2索 4索 ⑦筒 ⑦筒 七萬 五萬 六萬
Game 2 第7局 東家
   ツモ数: 2
   鳴き内容: 白 を大明槓した（白 白 白 白）
   直後の打牌: 中
   直後の手牌: 2索 2索 4索 4索 ⑨筒 ⑨筒 西 西 赤⑤筒
Game 2 第7局 東家
   ツモ数: 3
   鳴き内容: 西 をポンして 西

In [9]:
from mahjong_py.display_handflow import display_hand_flow_by_kyoku

display_hand_flow_by_kyoku(game2_str, game_index=2)




── Game 2 手配進行 ──
== 第1局 ==
配 牌:
  東家: 一萬 5索 八萬 八萬 南 白 白 七萬 七萬 9索 南 ⑤筒 二萬
  南家: 赤五萬 ⑨筒 八萬 三萬 東 二萬 ⑤筒 7索 6索 中 ⑧筒 ⑦筒 三萬
  西家: 4索 二萬 西 東 ⑥筒 2索 ⑤筒 3索 六萬 發 6索 7索 ⑥筒
  北家: ⑦筒 九萬 7索 8索 ①筒 二萬 赤5索 2索 1索 9索 六萬 ⑨筒 ⑧筒

ツモ:
  東家: 9索 8索 ④筒 4索 七萬 4索 三萬 東 6索 七萬 ①筒 西 北 五萬 西 赤⑤筒
  南家: ⑧筒 9索 六萬 ⑦筒 8索 北 八萬 三萬 北 ⑨筒 1索 3索 ⑥筒 中 發 ③筒
  西家: 四萬 ⑦筒 2索 九萬 ⑨筒 一萬 3索 ②筒 ③筒 五萬 九萬 中 東 5索 四萬
  北家: 四萬 8索 3索 發 發 ③筒 1索 九萬 ④筒 ④筒 一萬 ⑧筒 白 四萬

捨て牌:
  東家: ⑤筒 5索 ④筒 4索 8索 二萬 4索 三萬 東 6索 ①筒 一萬 北 五萬 9索 赤⑤筒
  南家: 中 東 9索 ⑤筒 八萬 二萬 八萬 北 北 三萬 1索 3索 ⑥筒 中 發 ③筒
  西家: 東 發 西 九萬 ⑨筒 一萬 二萬 ②筒 ③筒 五萬 九萬 中 東 5索 四萬
  北家: ①筒 九萬 二萬 發 發 ③筒 8索 九萬 ④筒 ④筒 一萬 1索 1索 赤5索
== 第2局 ==
配 牌:
  東家: ⑥筒 中 九萬 東 赤⑤筒 ⑥筒 ③筒 赤5索 發 北 南 中 6索
  南家: 西 ④筒 七萬 ④筒 3索 9索 ⑦筒 2索 ⑤筒 六萬 1索 6索 5索
  西家: 3索 ⑧筒 ⑨筒 三萬 ①筒 ⑦筒 九萬 2索 ①筒 ⑦筒 1索 ②筒 四萬
  北家: ⑤筒 ⑧筒 6索 白 五萬 五萬 三萬 六萬 北 ②筒 ②筒 4索 七萬

ツモ:
  東家: 八萬 發 七萬 西 ②筒
  南家: 三萬 西 ⑨筒 7索 八萬
  西家: 1索 二萬 7索 ④筒 一萬
  北家: 一萬 東 赤五萬 八萬 4索

捨て牌:
  東家: 南 東 北 西 ⑥筒
  南家: 西 西 三萬 ⑨筒 9索
  西家: 1索 二萬 7索 ④筒 一萬
  北家: 北 東 ⑧筒 白 一萬
== 第3局 ==
配 牌:
  東家: ②筒 ②筒 ③筒 ⑨筒 ①筒 北

In [10]:
from mahjong_py.display_agari_fixed import display_agari_details

display_agari_details(game2_str, game_index=2)




── Game 2 和了詳細 ──
Game 2 第1局 和了者: 南家
   メルド枚数: 2  隠し枚数: 12  合計: 14枚
   手牌: 三萬 三萬 四萬 赤五萬 六萬 ⑦筒 ⑦筒 ⑧筒 ⑧筒 ⑨筒 ⑨筒 6索 7索 8索
   役: 平和, 一盃口, 赤ドラ
   得点: 30,3900,0
Game 2 第2局 和了者: 南家
   メルド枚数: 2  隠し枚数: 12  合計: 14枚
   手牌: 六萬 七萬 八萬 ④筒 ④筒 ⑤筒 ⑥筒 ⑦筒 1索 2索 3索 5索 6索 7索
   役: 立直, 一発, ドラ, 裏ドラ
   得点: 40,12000,1
Game 2 第3局 和了者: 北家
   メルド枚数: 5  隠し枚数: 9  合計: 14枚
   手牌: 二萬 二萬 六萬 七萬 八萬 ⑦筒 ⑧筒 ⑨筒 1索 2索 3索
   役: 場風 東, ドラ
   得点: 30,2000,0
Game 2 第4局 和了者: 南家
   メルド枚数: 2  隠し枚数: 12  合計: 14枚
   手牌: 八萬 八萬 ②筒 ③筒 ④筒 ⑦筒 ⑧筒 ⑨筒 2索 3索 4索 白 白 白
   役: 門前清自摸和, 役牌 白
   得点: 40,2700,0
Game 2 第5局 和了者: 東家
   メルド枚数: 2  隠し枚数: 12  合計: 14枚
   手牌: 一萬 二萬 三萬 七萬 七萬 ④筒 赤⑤筒 ⑥筒 ⑥筒 ⑦筒 ⑧筒 5索 6索 7索
   役: 立直, 平和, ドラ, 赤ドラ, 裏ドラ
   得点: 30,8000,1
Game 2 第6局 和了者: 北家
   メルド枚数: 2  隠し枚数: 12  合計: 14枚
   手牌: 一萬 二萬 三萬 三萬 四萬 五萬 七萬 八萬 九萬 ④筒 ④筒 3索 4索 赤5索
   役: 立直, 平和, ドラ, 赤ドラ, 裏ドラ
   得点: 30,7700,0
Game 2 第7局 和了者: 東家
   メルド枚数: 11  隠し枚数: 3  合計: 14枚
   手牌: 2索 2索 2索 4索 4索
   役: 対々和, 役牌 白
   得点: 50,6400,0
Game 2 第8局 和了者: 東家
   メルド枚数: 2  隠し枚数: 12  合計: 14枚
   手牌: 一萬

In [11]:
from mahjong_py.display_reach_fixed import display_reach_info_fixed

display_reach_info_fixed(game2_str, game_index=2)




── Game 2 リーチ情報 ──
Game 2 第2局
   南家がリーチ宣言
   南家: リーチ時点 ツモ 5 回 / 打牌 4 回 [成立]
Game 2 第4局
   東家がリーチ宣言
   東家: リーチ時点 ツモ 15 回 / 打牌 14 回 [成立]
Game 2 第5局
   東家がリーチ宣言
   東家: リーチ時点 ツモ 7 回 / 打牌 6 回 [成立]
Game 2 第6局
   北家がリーチ宣言
   北家: リーチ時点 ツモ 10 回 / 打牌 9 回 [成立]
Game 2 第8局
   東家がリーチ宣言
   東家: リーチ時点 ツモ 10 回 / 打牌 9 回 [成立]


In [12]:
from mahjong_py.display_ryuukyoku import display_ryuukyoku_info

display_ryuukyoku_info(game2_str, game_index=2)




── Game 2 流局情報 ──
Game 2 第9局
   点数変動 (sc): 196,15,446,-15,54,-15,304,15
   東家の公開手牌: 二萬 三萬 四萬 六萬 七萬 八萬 ⑤筒 ⑥筒 ⑦筒 ⑦筒 ⑧筒 3索 3索
   北家の公開手牌: 七萬 八萬 九萬 ⑥筒 ⑧筒 3索 4索 5索 8索 8索


In [13]:
from mahjong_py.display_dora_fixed import display_dora_fixed

display_dora_fixed(game2_str, game_index=2)




── Game 2 ドラ表示牌 ──
第1局: 六萬 ②筒
第2局: 4索
第3局: 9索
第4局: 東 八萬
第5局: ⑤筒
第6局: 八萬
第7局: 三萬 ①筒
第8局: 南
第9局: 三萬 2索
第10局: 9索
